In [1]:
import numpy as np

In [3]:
X = np.array(range(50))
X[:5]

array([0, 1, 2, 3, 4])

In [4]:
k = 5 # time steps per sample
step = 1
Xy = [np.array(X[i-k:i+1], dtype=np.int) for i in range(k,len(X)-1,step)]
Xy = np.array(Xy)
Xy[:2]

array([[0, 1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5, 6]])

In [5]:
X, y = Xy[:,0:k], Xy[:,k]

In [6]:
# from http://philipperemy.github.io/keras-stateful-lstm/
def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return np.array(windows), np.array(windows_y)

In [7]:
prepare_sequences(X,y,window_length=4)

(array([[ 0,  1,  2,  3],
        [ 1,  2,  3,  4],
        [ 1,  2,  3,  4],
        [ 2,  3,  4,  5],
        [ 2,  3,  4,  5],
        [ 3,  4,  5,  6],
        [ 3,  4,  5,  6],
        [ 4,  5,  6,  7],
        [ 4,  5,  6,  7],
        [ 5,  6,  7,  8],
        [ 5,  6,  7,  8],
        [ 6,  7,  8,  9],
        [ 6,  7,  8,  9],
        [ 7,  8,  9, 10],
        [ 7,  8,  9, 10],
        [ 8,  9, 10, 11],
        [ 8,  9, 10, 11],
        [ 9, 10, 11, 12],
        [ 9, 10, 11, 12],
        [10, 11, 12, 13],
        [10, 11, 12, 13],
        [11, 12, 13, 14],
        [11, 12, 13, 14],
        [12, 13, 14, 15],
        [12, 13, 14, 15],
        [13, 14, 15, 16],
        [13, 14, 15, 16],
        [14, 15, 16, 17],
        [14, 15, 16, 17],
        [15, 16, 17, 18],
        [15, 16, 17, 18],
        [16, 17, 18, 19],
        [16, 17, 18, 19],
        [17, 18, 19, 20],
        [17, 18, 19, 20],
        [18, 19, 20, 21],
        [18, 19, 20, 21],
        [19, 20, 21, 22],
        [19,

# Stateful LSTM to learn one-char to one-char mapping

From https://machinelearningmastery.com/understanding-stateful-lstm-recurrent-neural-networks-python-keras/

In [16]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
# fix random seed for reproducibility
numpy.random.seed(7)
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# prepare the dataset of input to output pairs encoded as integers
seq_length = 1
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
	seq_in = alphabet[i:i + seq_length]
	seq_out = alphabet[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
	#print(seq_in, '->', seq_out)
dataX = np.array(dataX)
dataY = np.array(dataY)

In [18]:
dataX.shape, dataY.shape

((25, 1), (25,))

In [20]:
# reshape X to be [samples, time steps, features]
X = dataX.reshape(len(dataX), seq_length, 1)
dataX.shape

(25, 1)

In [21]:
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
y.shape

(25, 26)

In [ ]:
# create and fit the model
batch_size = 1
model = Sequential()
model.add(LSTM(50, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
for i in range(300):
	model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()
# summarize performance of the model
scores = model.evaluate(X, y, batch_size=batch_size, verbose=0)
model.reset_states()
print("Model Accuracy: %.2f%%" % (scores[1]*100))
# demonstrate some model predictions
seed = [char_to_int[alphabet[0]]]
for i in range(0, len(alphabet)-1):
	x = numpy.reshape(seed, (1, len(seed), 1))
	x = x / float(len(alphabet))
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	print(int_to_char[seed[0]], "->", int_to_char[index])
	seed = [index]
model.reset_states()
# demonstrate a random starting point
letter = "K"
seed = [char_to_int[letter]]
print("New start: ", letter)
for i in range(0, 5):
	x = numpy.reshape(seed, (1, len(seed), 1))
	x = x / float(len(alphabet))
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	print(int_to_char[seed[0]], "->", int_to_char[index])
	seed = [index]
model.reset_states()